Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Tutorial: Load NOAA ISD Weather Data

## Install azureml-opendatasets SDK

In [1]:
!pip uninstall -y azureml-contrib-opendatasets
!pip install azureml-contrib-opendatasets

Uninstalling azureml-contrib-opendatasets-1.0.30:
  Successfully uninstalled azureml-contrib-opendatasets-1.0.30
  Using cached https://files.pythonhosted.org/packages/64/51/4d3de57cf210941346d907584e0e6e56780067bc3555250b1fe62c2285f7/azureml_contrib_opendatasets-1.0.30-py3-none-any.whl


Import NoaaIsdWeather class from azureml-opendatasets

In [2]:
# This is a contrib package in preview. The package name is subject to change.

from azureml.contrib.opendatasets import NoaaIsdWeather

from datetime import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta

> 1. Set start_date and end_date.
> 2. New an instance of NoaaIsdWeather, disable telemetry if you don't want to send logs to Azure.
> 3. Call to_pandas_dataframe() method to get a pandas DataFrame for the last month of import datetime. We only retrieve the last month of the data due to the performance and memory size consideration.

In [3]:
start_date = parser.parse('2018-1-1')
end_date = parser.parse('2018-1-31')
isd = NoaaIsdWeather(start_date, end_date, enable_telemetry=False)
isd.to_pandas_dataframe().info()
print('isd done')

Target paths: ['/year=2018/month=1/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading ISDWeather/year=2018/month=1/part-00043-tid-8746345262315685594-9d96c040-809c-4da3-a12e-649c3f8f2505-77900.c000.snappy.parquet under container isdweatherdatacontainer
Done.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11190325 entries, 0 to 11543564
Data columns (total 22 columns):
usaf                       object
wban                       object
datetime                   datetime64[ns]
latitude                   float64
longitude                  float64
elevation                  float64
windAngle                  float64
windSpeed                  float64
temperature                float64
seaLvlPressure             float64
cloudCoverage              object
presentWeatherIndicator    float64
pastWeatherIndicator       float64
precipTime                 float64
precipDepth                float64
snowDepth                  float64
stationName                object
c